In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import scipy
import os
import requests
from dotenv import load_dotenv
import json

from opencage.geocoder import OpenCageGeocode
%matplotlib inline

In [3]:
# Load .env enviroment variables
load_dotenv()

True

In [4]:
# Set GeoCode API key
geocoder_key = os.getenv('OPEN_CAGE_API')
type(geocoder_key)

str

In [5]:
# Feed the API key into the imported OpenCageGeocode
geocoder = OpenCageGeocode(geocoder_key)

In [13]:
# Bring in the Area csv file
arena_csv = Path('Cleansed_csv/arena_location.csv')
# Read csv into a pandas Dataframe
arena = pd.read_csv(arena_csv)
arena = arena[["Team Name","Arena Location","lat","long"]]
arena

,Team Name,Arena Location,lat,long
0,Atlanta Hawks,"Atlanta, Georgia",33.748992,-84.390264
1,Boston Celtics,"Boston, Massachusetts",42.360253,-71.058291
2,Brooklyn Nets,"Brooklyn, New York",40.650102,-73.949583
3,Charlotte Hornets,"Charlotte, North Carolina",35.227209,-80.843083
4,Chicago Bulls,"Chicago, Illinois",41.875562,-87.624421
5,Cleveland Cavaliers,"Cleveland, Ohio",41.505161,-81.693445
6,Dallas Mavericks,"Dallas, Texas",32.776272,-96.796856
7,Denver Nuggets,"Denver, Colorado",39.739236,-104.984862
8,Detroit Pistons,"Detroit, Michigan",42.331551,-83.046640
9,Golden State Warriors,"San Francisco, California",37.779026,-122.419906


In [14]:
# Bring in salary average csv file
salary_csv = Path('Cleansed_csv/salary_avg.csv')
# Read csv into a pandas Dataframe
salary = pd.read_csv(salary_csv)
salary

,Team,Salary w/ Inflation
0,Atlanta Hawks,81131763.5
1,Boston Celtics,91050095.7
2,Brooklyn Nets,86496264.2
3,Charlotte Bobcats,81672887.6
4,Chicago Bulls,82796444.4
5,Cleveland Cavaliers,96250231.9
6,Dallas Mavericks,93516389.1
7,Denver Nuggets,80095618.7
8,Detroit Pistons,85159538.0
9,Golden State Warriors,89601170.0


In [20]:
#Concat data in preparation for mapping
arena_salaries = pd.concat([arena, salary], axis=1, sort=True)
arena_salaries = arena_salaries.drop(columns=['Team'])
arena_salaries

,Team Name,Arena Location,lat,long,Salary w/ Inflation
0,Atlanta Hawks,"Atlanta, Georgia",33.748992,-84.390264,81131763.5
1,Boston Celtics,"Boston, Massachusetts",42.360253,-71.058291,91050095.7
2,Brooklyn Nets,"Brooklyn, New York",40.650102,-73.949583,86496264.2
3,Charlotte Hornets,"Charlotte, North Carolina",35.227209,-80.843083,81672887.6
4,Chicago Bulls,"Chicago, Illinois",41.875562,-87.624421,82796444.4
5,Cleveland Cavaliers,"Cleveland, Ohio",41.505161,-81.693445,96250231.9
6,Dallas Mavericks,"Dallas, Texas",32.776272,-96.796856,93516389.1
7,Denver Nuggets,"Denver, Colorado",39.739236,-104.984862,80095618.7
8,Detroit Pistons,"Detroit, Michigan",42.331551,-83.046640,85159538.0
9,Golden State Warriors,"San Francisco, California",37.779026,-122.419906,89601170.0


In [18]:
import folium

map = folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles="Stamen Terrain")
map

In [21]:
for idx in arena_salaries.index:
    row = arena_salaries.loc[idx]
    lat, long = row.lat, row.long
    folium.CircleMarker(
        location=[lat, long],
        clustered_marker = False,
        radius=arena_salaries.iloc[idx]['Salary w/ Inflation']/10000000,
        draggable = True,
        color="blue",
        popup = str(row["Team Name"]) + ", " + str(row["Salary w/ Inflation"]),
        tooltip = row["Arena Location"],
        fill_color="red"
        ).add_to(map)
map